# Processing pupil size data and epoching

When working with event-based recordings, it's often useful to extract and analyze data segments surrounding those events, rather than processing the entire recording. In mobile eye-tracking, such events could include stimulus presentations or the participant entering a specific area. In this tutorial, we'll demonstrate how to epoch data around these events using the `pyneon.Epochs` class, with the sample recording `screenFlash`.

In [ ]:
from pyneon import Dataset, Recording, get_sample_data

# Create a Recording object
# dataset_dir = get_sample_data("screenFlash") / "Timeseries Data + Scene Video"
dataset_dir = (
    get_sample_data("PLR") / "Native Recording Data"
)  # Can also use native data
dataset = Dataset(dataset_dir)
rec = dataset.recordings[0]
scene_video = rec.scene_video
blinks = rec.blinks
events = rec.events
eye_states = rec.eye_states

This recording features a participant observing a screen that flashes from black to white (flash duration = 0.75 s), with a 3-second interval between flashes. We expect the pupil to constrict in response to each flash, making this dataset ideal for demonstrating epoching. Here is how the flash appears to the participant:

In [ ]:
import matplotlib.pyplot as plt

# Create a figure
fig, axs = plt.subplots(1, 2, figsize=(8, 3))

# Visualize two frames from the video that correspond to the dark and flash periods
scene_video.plot_frame(100, ax=axs[0], auto_title=False, show=False)
scene_video.plot_frame(120, ax=axs[1], auto_title=False, show=False)
axs[0].set_title("Dark")
axs[1].set_title("Flash")
plt.show()

In [ ]:
# Filter events to get flash onsets
flashes = events.filter_by_name("flash onset")
print(flashes.data)

# # Compute video frame intensities
# intensities = scene_video.compute_intensity()
# plt.figure(figsize=(8, 3))
# plt.plot(intensities["intensity"])
# for onset in flashes.start_ts:
#     dur = plt.axvspan(
#         onset, onset + 0.75 * 1e9, color="lightgray", label="Flashes"
#     )  # Each flash lasts 2 seconds
# plt.legend(handles=[dur])

## Visualizing raw pupil size data and repairing blink artifacts

Let's inspect the raw pupil size data along with blink events. The shaded region shows when the screen is bright; dashed lines indicate the onset of blinks.

In [ ]:
plt.figure(figsize=(8, 3))

# Visualize the flashes
for flash_start in flashes.start_ts:
    dur = plt.axvspan(
        flash_start, flash_start + 0.75 * 1e9, color="lightgray", label="Flashes"
    )  # Each flash lasts 2 seconds

# Visualize the blinks
for blink_start in blinks.start_ts:
    line = plt.axvline(blink_start, color="k", linestyle="--", lw=1, label="Blinks")

# Plot the pupil data
(pupil_l,) = plt.plot(eye_states["pupil diameter left [mm]"])

plt.xlabel("Timestamp [ns]")
plt.ylabel("Left pupil diameter [mm]")
plt.legend(handles=[dur, line])
plt.show()

From the plot we can note:
- Pupil size decreases promptly after each change in screen brightness.
- The participant frequently blinks in response to flashes, and blinks introduce distinct transient artifacts in the pupil size measurement.
 
First, we’ll address blink artifacts using the `interpolate_events()` method of the `Stream` class. This method masks the data during events (in this case, blinks) and interpolates values around them, effectively removing the transient artifacts.

In [ ]:
eye_states.interpolate_events(blinks, inplace=True)

plt.figure(figsize=(8, 3))

# Visualize the flashes
for flash_start in flashes.start_ts:
    dur = plt.axvspan(
        flash_start, flash_start + 7.5 * 1e8, color="lightgray", label="Flashes"
    )  # Each flash lasts 2 seconds

# Plot the pupil data
(pupil_l,) = plt.plot(eye_states["pupil diameter left [mm]"])

plt.xlabel("Timestamp [ns]")
plt.ylabel("Left pupil diameter [mm]")
plt.legend(handles=[dur])
plt.show()

## Defining Epochs: Creating the `epochs_info` DataFrame

In order to perform epoching, we require information about the timing and length of epochs. Explicitly, every epoch requires:
- A reference time `t_ref`
- A time window defined by `t_before` and `t_after` to extract the data from
- An event `description` to annotate the epoch.

PyNeon's `Epochs` implementation allows epochs of variable lengths, where `t_before` and `t_after` may differ for each event. As a result, defining all epochs requires a DataFrame (`epochs_info`) with columns: `t_ref`, `t_before`, `t_after`, and `description`.

While constructing such a DataFrame by hand is possible, it's easier to generate one using event information from PyNeon `Events` objects. The `events_to_epochs_info` function streamlines this process by using the onsets of events and user-defined `t_before`/`t_after` values.

In this example, we’ll use `"flash onset"` events, extracting 0.5 seconds before to 3 seconds after each event; the description is set to the event name automatically.

In [ ]:
from pyneon import Epochs, events_to_epochs_info

# For complex events, such as those from `events.csv`, we need to specify the event name
flash_epochs_info = events_to_epochs_info(
    events, t_before=0.5, t_after=3, t_unit="s", event_name="flash onset"
)

# Print the epoching-ready epochs_info
print("\nFlash times DataFrame:")
print(flash_epochs_info.head())

## Creating `Epochs` objects

With the `epochs_info` DataFrame, we can create epochs by passing it along with the relevant `Stream` or `Events` data to the `Epochs` class.

In this example, we create two types of epochs to test two hypotheses:
- `eye_ep`: epochs from `eye_states` (a `Stream` object), containing pupil size. We hypothesize pupil size will decrease after each flash.
- `blink_ep`: epochs from `blinks` (an `Events` object), letting us check if blinks cluster after each flash.

In [ ]:
eye_ep = Epochs(eye_states, flash_epochs_info)
blink_ep = Epochs(blinks, flash_epochs_info)
print(f"{len(eye_ep)} epochs created from {len(flash_epochs_info)} events")

Given that we have very sparse blink events, it is not surprising that PyNeon emits a warning when creating the `blink_ep` epochs.

The `Epochs` class provides two key attributes (For details, see the [API reference](https://ncc-brain.github.io/PyNeon/reference/epochs.html)):
- `epochs`: a nested DataFrame, indexed by epoch number, with data for each epoch stored under the `data` column.
- `data`: (not to be confused with the `data` column in `Epochs.epochs`) a DataFrame in the same structure as the source, but annotated with epoch membership and timing information (`epoch index`, `epoch time`, and `epoch description`). As a flattened version of `Epochs.epochs`, `Epochs.data` may contain overlapping epochs; in such cases, the additional columns will only show information from the most recent epoch a time point belongs to. Therefore, we recommend primarily using `Epochs.epochs` for analysis.

In [ ]:
epochs = eye_ep.epochs_dict
print(f"epochs is a {type(epochs)} with {len(epochs)} items.")
first_epoch = epochs[0]
print(f"Dict values are {type(first_epoch)} with shape {first_epoch.data.shape}.")

annot = eye_ep.annotate_source()
epochs_info = eye_ep.epochs_info
print(epochs_info.index)

## Visualizing epoch data

The `plot()` method provides a convenient way to visualize data across epochs.

For `Epochs` created from a `Stream` (like eye states), specify the desired column to plot all across epochs in a single figure, colored by epoch index. We can observe from the following plot that pupil restriction occurs after each flash consistently, except for the first flash, likely due to a different baseline pupil size.

In [ ]:
fig, ax = plt.subplots(figsize=(8, 3))
fig, ax = eye_ep.plot("pupil diameter left [mm]", ax=ax)

For `Events`-based `Epochs` (like blinks), simply call `plot()` with no arguments. Here, each event is displayed as a horizontal line spanning its duration, vertically sorted by epoch. We can see that blinks (N=4 in this example) tend to happen 1 to 1.5 seconds after each flash.

In [ ]:
fig, ax = blink_ep.plot()

## Converting `Stream`-based `Epochs` to a Numpy array

Rather than viewing epochs as nested DataFrames, under the right conditions, you can convert an `Epochs` object to a 3D NumPy array useful for time-series analyses (like with [MNE-Python](https://mne.tools/stable/generated/mne.Epochs.html)).  

For this:
- The source must be a uniformly sampled `Stream` (constant intervals between samples).
- All epochs must have the same durations (`t_before` and `t_after` equal).

If these are not met, PyNeon will raise a `ValueError` when `to_numpy()` is called.

To ensure compatibility, interpolate the `Stream` data prior to epoching, and maintain constant window sizes across epochs. The resulting array will be shaped `(n_epochs, n_channels, n_times)`.

In [ ]:
# Create an epoch with the interpolated data
epochs_np, info = eye_ep.to_numpy()

print(f"(n_epochs, n_channels, n_times) = {epochs_np.shape}")
print(info)

In this case, we have 10 epochs, 20 data channels, and 699 time steps—consistent with the 200 Hz sampling rate and the 3.5-second epoch window. The `info` object contains lists of column IDs and epoch times. Now, the grand average is easily computed:

In [ ]:
# Average the first channel (Pupil diameter left [mm]) across all epochs
epoch_times = info["epoch_times"] / 1e9
pupil_mean = epochs_np[:, 0, :].mean(axis=0)
pupil_std = epochs_np[:, 0, :].std(axis=0)

plt.figure(figsize=(8, 4))

# Show a vertical line at 0
plt.axvline(x=0, color="k", linestyle="--")
# Plot the mean pupil diameter and standard deviation
plt.plot(epoch_times, pupil_mean)
plt.fill_between(epoch_times, pupil_mean - pupil_std, pupil_mean + pupil_std, alpha=0.2)
plt.xlim(-0.5, 3)
plt.xlabel("flash onset time [s]")
plt.ylabel("Pupil Diameter left [mm]")
plt.show()

## Baseline correction

Optionally, we can also perform `baseline_correction()` on the data to address global fluctuations in data. approach is to subtract the mean of a pre-event baseline (`method="mean"`). If there exists a linear trend in the baseline that extends into the entire epoch, `method="linear"` will fit and remove it.

Here, we apply `baseline_correction()` with `method="mean"` to compare pupil size relative to a 500 ms pre-flash baseline, centering pre-flash values around zero and reducing variance.

In [ ]:
# Perform baseline correction
epochs_interp.baseline_correction(inplace=True, baseline=(-0.5, 0), method="mean")
epochs_np, info = epochs_interp.to_numpy()

# Average the first channel (Pupil diameter left [mm]) across all epochs
pupil_mean = epochs_np[:, 0, :].mean(axis=0)
pupil_std = epochs_np[:, 0, :].std(axis=0)

plt.figure(figsize=(8, 4))

# Show a vertical line at 0
plt.axvline(x=0, color="k", linestyle="--")
# Plot the mean pupil diameter and standard deviation
plt.plot(info["epoch_times"], pupil_mean)
plt.fill_between(
    info["epoch_times"], pupil_mean - pupil_std, pupil_mean + pupil_std, alpha=0.2
)
plt.xlim(-0.5, 3)
plt.xlabel("flash onset time [s]")
plt.ylabel("Pupil Diameter left [mm]")
plt.show()